In [106]:
import pandas as pd
import numpy as np
import altair as alt
from observable import Observable
import warnings
warnings.filterwarnings('ignore')

# Long-term Mobility Change after COVID in Puget Sound Region

## Dataset: COVID-19 Mobility Survey

The data comes from an online survey designed by the University of Washington (UW) researchers that collected data on stated travel behaviors before and during the pandemic, as well as expected travel choice for post-pandemic. Socio demographics, the use of information communication technologies (ICT), and attitudes towards different transportation modes were also collected. The survey was advertised through the Facebook page of UW Civil and Environmental Engineering Department and was live for 14 consecutive days in June-July 2020). Ads were run on Facebook, Instagram, Messenger, and other social media platforms owned by Facebook, and were set to be shown only to residents of the Puget Sound region. The ads reached 49,146 people, of whom 1,389 completed the survey (2.8%). There were 1,310 valid responses after data cleaning. More details >> https://digital.lib.washington.edu/researchworks/handle/1773/46655  

A glimpse of the dataset is shown as below:

In [2]:
pd.set_option('display.max_columns',None)

COVID_data = pd.read_csv("COVID_data.csv")

COVID_data.head(5)

,Which.of.the.following.best.describes.your.current.job.situation.,Are.you.employed.in.one.of.the.following.industries.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......work.from.home..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......shop.in.stores.for.groceries..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......order.groceries.online.for.pick.up.delivery..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......shop.in.stores.for.other.items..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......order.other.items.online.for.pick.up.delivery..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......eat.a.meal.in.a.restaurant..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......order.food.for.pick.up.delivery.from.a.restaurant..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Drive.alone.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Carpool.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Public.Transit..Bus..Light.Rail..Streetcar..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Ferry.or.Water.Taxi.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Solo.Ridehail..UberX..Lyft..or.Taxi.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Shared.Ridehail..e.g..UberPool..Lyft.Shared..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Bike.or.Scooter.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Shared.bike.or.scooter..Lime..JUMP..etc...,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Walk.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...what.mode.of.transportation.did.you.generally.use.for.the.following.purposes...Work.School.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...what.mode.of.transportation.did.you.generally.use.for.the.following.purposes...Grocery.Shopping.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...what.mode.of.transportation.did.you.generally.use.for.the.following.purposes...Other.Shopping.,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...what.mode.of.transportation.did.you.generally.use.for.the.following.purposes...Errands..e.g..bank..doctor.appointment..,In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...what.mode.of.transportation.did.you.generally.use.for.the.following.purposes...Social..Meal.or.Recreation.,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......work.from.home..,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......shop.in.stores.for.groceries..,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......order.groceries.online.for.pick.up.delivery..,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......shop.in.stores.for.other.items..,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......order.other.items.online.for.pick.up.delivery..,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......eat.a.meal.in.a.restaurant..,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......order.food.for.pick.up.delivery.from.a.restaurant..,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Drive.Alone.,In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.followi

## Analysis Questions

In this assignment, I'll start with exploring the physical and virtual mobility charactersitics before, during, and after COVID-19 (Q1 & Q2). And then I'll focus on the transit users (Q3 & Q4).    
1. How do people change their physical mobility (e.g., private car, transit) during, and even after COVID-19?
2. How do people change their virtual mobility (e.g., work from home) during, and even after COVID-19?
3. Are there interesting patterns in transit use in terms of transit use frequency before, during, and after COVID-19? Particularly, is there a "lost transit user" group who are reluctant to return to transit when the COVID-19 is over?
4. For "Lost transit users", are they more likely to turn to private cars, or working from home?

### Question 1: How do people change their physical mobility during and after COVID-19?

### Physical travel: transit

In normal times, almost half of survey respondents did not use transit. An obvious change is during COVID-19, when over 1,200 out of 1,310 are not using transit. However, we also see people change their expected transit use frequency after the pandemic. 

In [107]:
## Transit
## Before COVID
source = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Public.Transit..Bus..Light.Rail..Streetcar.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Transit use before COVID', 'Frequency']
Transit_before = source.groupby("Transit use before COVID").size().reset_index(name='count')

## During COVID
source = COVID_data[["In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Public.Transit..Bus..Light.Rail..Streetcar.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Transit use during COVID', 'Frequency']
Transit_during = source.groupby("Transit use during COVID").size().reset_index(name='count')

## After COVID
source = COVID_data[["Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will.use.the.following.modes.of.transportation...Public.Transit..Bus..Light.Rail..Streetcar.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Transit use after COVID', 'Frequency']
Transit_after = source.groupby("Transit use after COVID").size().reset_index(name='count')


In [167]:
Transit_0 = alt.Chart(Transit_before).mark_bar().encode(
    x = alt.X('Transit use before COVID:O', 
        sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='Before COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title='Count'),
        scale = alt.Scale(domain=[0,1350]),
             )
).properties(width=250, height = 180)

text_0 = Transit_0.mark_text(
    align='center',
    baseline='top',
    dy=-15  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count:Q'
)

Transit_1 = alt.Chart(Transit_during).mark_bar().encode(
    x = alt.X('Transit use during COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='During COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1350]))
).properties(width=250, height = 180)

text_1 = Transit_1.mark_text(
    align='center',
    baseline='top',
    dy=-15  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count:Q'
)

Transit_2 = alt.Chart(Transit_after).mark_bar().encode(
    x = alt.X('Transit use after COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='After COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1350]))
).properties(width=250, height = 180)

text_2 = Transit_2.mark_text(
    align='center',
    baseline='top',
    dy=-15  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count:Q'
)

Transit_0 + text_0 | Transit_1 + text_1 | Transit_2 + text_2

alt.HConcatChart(...)

### Physical travel: drive alone

In [5]:
## Before COVID
source = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Drive.alone."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Private car before COVID', 'Frequency']
Driving_before = source.groupby("Private car before COVID").size().reset_index(name='count')

## During COVID
source = COVID_data[["In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Drive.Alone."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Private car during COVID', 'Frequency']
Driving_during = source.groupby("Private car during COVID").size().reset_index(name='count')

## After COVID
source = COVID_data[["Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will.use.the.following.modes.of.transportation...Drive.Alone."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Private car after COVID', 'Frequency']
Driving_after = source.groupby("Private car after COVID").size().reset_index(name='count')

In [169]:
Driving_0 = alt.Chart(Driving_before).mark_bar().encode(
    x = alt.X('Private car before COVID:O', 
        sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='Before COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title='Count'),
        scale = alt.Scale(domain=[0,800]))
).properties(width=250, height = 180)

text_0 = Driving_0.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

Driving_1 = alt.Chart(Driving_during).mark_bar().encode(
    x = alt.X('Private car during COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='During COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,800]))
).properties(width=250, height = 180)

text_1 = Driving_1.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

Driving_2 = alt.Chart(Driving_after).mark_bar().encode(
    x = alt.X('Private car after COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='After COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,800]))
).properties(width=250, height = 180)

text_2 = Driving_2.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

Driving_0 + text_0 | Driving_1 + text_1 | Driving_2 + text_2

alt.HConcatChart(...)

### Physical travel: shared ride hailing

In [7]:
## Before COVID
source = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Shared.Ridehail..e.g..UberPool..Lyft.Shared.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['SharedRH before COVID', 'Frequency']
SharedRH_before = source.groupby("SharedRH before COVID").size().reset_index(name='count')

## During COVID
source = COVID_data[["In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Shared.Ridehail..e.g..UberPool..Lyft.Shared.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['SharedRH during COVID', 'Frequency']
SharedRH_during = source.groupby("SharedRH during COVID").size().reset_index(name='count')

## After COVID
source = COVID_data[["Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will.use.the.following.modes.of.transportation...Shared.Ridehail..e.g..UberPool..Lyft.Shared.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['SharedRH after COVID', 'Frequency']
SharedRH_after = source.groupby("SharedRH after COVID").size().reset_index(name='count')

In [173]:
SharedRH_0 = alt.Chart(SharedRH_before).mark_bar().encode(
    x = alt.X('SharedRH before COVID:O', 
        sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='Before COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title='Count'),
        scale = alt.Scale(domain=[0,1450]))
).properties(width=250, height = 180)

text_0 = SharedRH_0.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

SharedRH_1 = alt.Chart(SharedRH_during).mark_bar().encode(
    x = alt.X('SharedRH during COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='During COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1450]))
).properties(width=250, height = 180)

text_1 = SharedRH_1.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

SharedRH_2 = alt.Chart(SharedRH_after).mark_bar().encode(
    x = alt.X('SharedRH after COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='After COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1450]))
).properties(width=250, height = 180)

text_2 = SharedRH_2.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

SharedRH_0 + text_0 | SharedRH_1 + text_1 | SharedRH_2 + text_2

alt.HConcatChart(...)

### Physical travel: biking

In [9]:
## Before COVID
source = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Bike.or.Scooter."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['PBiking before COVID', 'Frequency']
PBiking_before = source.groupby("PBiking before COVID").size().reset_index(name='count')

## During COVID
source = COVID_data[["In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Bike.or.Scooter."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['PBiking during COVID', 'Frequency']
PBiking_during = source.groupby("PBiking during COVID").size().reset_index(name='count')

## After COVID
source = COVID_data[["Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will.use.the.following.modes.of.transportation...Bike.or.Scooter."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['PBiking after COVID', 'Frequency']
PBiking_after = source.groupby("PBiking after COVID").size().reset_index(name='count')

In [174]:
PBiking_0 = alt.Chart(PBiking_before).mark_bar().encode(
    x = alt.X('PBiking before COVID:O', 
        sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='Before COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title='Count'),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_0 = PBiking_0.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

PBiking_1 = alt.Chart(PBiking_during).mark_bar().encode(
    x = alt.X('PBiking during COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='During COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_1 = PBiking_1.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

PBiking_2 = alt.Chart(PBiking_after).mark_bar().encode(
    x = alt.X('PBiking after COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='After COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_2 = PBiking_2.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

PBiking_0 + text_0 | PBiking_1 + text_1 | PBiking_2 + text_2

alt.HConcatChart(...)

### Physical travel: walking

In [11]:
## Before COVID
source = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Walk."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Walking before COVID', 'Frequency']
Walking_before = source.groupby("Walking before COVID").size().reset_index(name='count')

## During COVID
source = COVID_data[["In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Walk."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Walking during COVID', 'Frequency']
Walking_during = source.groupby("Walking during COVID").size().reset_index(name='count')

## After COVID
source = COVID_data[["Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will.use.the.following.modes.of.transportation...Walk."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['Walking after COVID', 'Frequency']
Walking_after = source.groupby("Walking after COVID").size().reset_index(name='count')

In [175]:
Walking_0 = alt.Chart(Walking_before).mark_bar().encode(
    x = alt.X('Walking before COVID:O', 
        sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='Before COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title='Count'),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_0 = Walking_0.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

Walking_1 = alt.Chart(Walking_during).mark_bar().encode(
    x = alt.X('Walking during COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='During COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_1 = Walking_1.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

Walking_2 = alt.Chart(Walking_after).mark_bar().encode(
    x = alt.X('Walking after COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='After COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_2 = Walking_2.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

Walking_0 + text_0 | Walking_1 + text_1 | Walking_2 + text_2

alt.HConcatChart(...)

### Question 2: How do people change their virtual mobility during and after COVID-19?

### Virtual mobility: WFH (work from home)

As expected, most of the respondents did not work from home before the pandemic. During the pandemic, only 409 people still never work from home, while over half of the respondents began working from home everyday. After the pandemic, we could see the anticipated work from home frequency is higher than before the pandemic.

In [13]:
## Before COVID
source = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......work.from.home.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['WFH before COVID', 'Frequency']
WFH_before = source.groupby("WFH before COVID").size().reset_index(name='count')

## During COVID
source = COVID_data[["In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......work.from.home.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['WFH during COVID', 'Frequency']
WFH_during = source.groupby("WFH during COVID").size().reset_index(name='count')

## After COVID
source = COVID_data[["Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will......work.from.home.."]]
source.loc[:,'ID'] = source.reset_index().index
source.columns = ['WFH after COVID', 'Frequency']
WFH_after = source.groupby("WFH after COVID").size().reset_index(name='count')

In [177]:
WFH_0 = alt.Chart(WFH_before).mark_bar().encode(
    x = alt.X('WFH before COVID:O', 
        sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='Before COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title='Count'),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_0 = WFH_0.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

WFH_1 = alt.Chart(WFH_during).mark_bar().encode(
    x = alt.X('WFH during COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='During COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_1 = WFH_1.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

WFH_2 = alt.Chart(WFH_after).mark_bar().encode(
    x = alt.X('WFH after COVID:O', sort=['Never', 'Once a month or less', 'A few times a month', '1-2 days a week', '3-4 days a week', 'Everyday'],
        axis=alt.Axis(title='After COVID')),
    y = alt.Y('count:Q',
        axis=alt.Axis(title=''),
        scale = alt.Scale(domain=[0,1200]))
).properties(width=250, height = 180)

text_2 = WFH_2.mark_text(
    align='center',
    baseline='top',
    dy=-15
).encode(
    text='count:Q'
)

WFH_0 + text_0 | WFH_1 + text_1 | WFH_2 + text_2

alt.HConcatChart(...)

### Question 3: Are there interesting patterns in transit use in terms of transit use frequency before, during, and after COVID-19?   
Here I applied a exploratory clustering approach using transit use frequency before, during, and after COVID-19. We also change the orginal Likert-scale travel frequency to a quantitative one. As a result, a total of 4 groups are identified. 
+ **Lost transit users**: Lost transit users describe the group who used transit frequently before the pandemic, but would not return to transit after the pandemic.
+ **Rebounding moderate transit users**: Rebounding moderate transit users describe the group who used transit at a moderate level before the pandemic, and would return to transit after COVID-19.
+ **Rebounding frequent transit users**: Similar to rebounding moderate users, rebounding frequent users have high transit use frequency before the pandemic, and would return to high transit use frequency after the pandemic.
+ **Non transit users**: Non-transit users did not use transit most of the times, nor would they in the future.

In [108]:
from sklearn.cluster import KMeans

Transit = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Public.Transit..Bus..Light.Rail..Streetcar..",
                     "In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Public.Transit..Bus..Light.Rail..Streetcar..",
                     "Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will.use.the.following.modes.of.transportation...Public.Transit..Bus..Light.Rail..Streetcar.."]]
Transit.columns = ["Before COVID", "During COVID", "After COVID"]

Transit.loc[(Transit.loc[:,"Before COVID"] == 'Never'),'Before COVID'] = 0
Transit.loc[(Transit.loc[:,"Before COVID"] == 'Once a month or less'),'Before COVID'] = 0.125
Transit.loc[(Transit.loc[:,"Before COVID"] == 'A few times a month'),'Before COVID'] = 0.625
Transit.loc[(Transit.loc[:,"Before COVID"] == '1-2 days a week'),'Before COVID'] = 1.5
Transit.loc[(Transit.loc[:,"Before COVID"] == '3-4 days a week'),'Before COVID'] = 3.5
Transit.loc[(Transit.loc[:,"Before COVID"] == 'Everyday'),'Before COVID'] = 7

Transit.loc[(Transit.loc[:,"During COVID"] == 'Never'),'During COVID'] = 0
Transit.loc[(Transit.loc[:,"During COVID"] == 'Once a month or less'),'During COVID'] = 0.125
Transit.loc[(Transit.loc[:,"During COVID"] == 'A few times a month'),'During COVID'] = 0.625
Transit.loc[(Transit.loc[:,"During COVID"] == '1-2 days a week'),'During COVID'] = 1.5
Transit.loc[(Transit.loc[:,"During COVID"] == '3-4 days a week'),'During COVID'] = 3.5
Transit.loc[(Transit.loc[:,"During COVID"] == 'Everyday'),'During COVID'] = 7

Transit.loc[(Transit.loc[:,"After COVID"] == 'Never'),'After COVID'] = 0
Transit.loc[(Transit.loc[:,"After COVID"] == 'Once a month or less'),'After COVID'] = 0.125
Transit.loc[(Transit.loc[:,"After COVID"] == 'A few times a month'),'After COVID'] = 0.625
Transit.loc[(Transit.loc[:,"After COVID"] == '1-2 days a week'),'After COVID'] = 1.5
Transit.loc[(Transit.loc[:,"After COVID"] == '3-4 days a week'),'After COVID'] = 3.5
Transit.loc[(Transit.loc[:,"After COVID"] == 'Everyday'),'After COVID'] = 7

X = np.array(Transit)
Transit_kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

Transit_cluster = pd.concat([Transit, pd.Series(Transit_kmeans.labels_)], axis=1)
Transit_cluster.columns = ["COVID-before", "COVID-during", "COVID-after", "Cluster"]

Transit_cluster['id'] = Transit_cluster.reset_index().index
Transit_cluster_melt = pd.melt(frame=Transit_cluster, 
        id_vars=["id", "Cluster"], 
        var_name="Time", 
        value_name="Freq")

In [184]:
from altair.expr import datum

Transit_cluster_melt.loc[(Transit_cluster_melt.Cluster == 0),'Cluster']='Rebounding moderate users'
Transit_cluster_melt.loc[(Transit_cluster_melt.Cluster == 1),'Cluster']='Rebounding frequent users'
Transit_cluster_melt.loc[(Transit_cluster_melt.Cluster == 2),'Cluster']='Lost transit users'
Transit_cluster_melt.loc[(Transit_cluster_melt.Cluster == 3),'Cluster']='Non-transit users'

alt.Chart(Transit_cluster_melt).transform_window(
    index='count()'
).transform_fold(
    ['Before COVID', 'During COVID', 'After COVID']
).mark_line().encode(
    alt.X('Time:O', 
        sort=['Before COVID', 'During COVID', 'After COVID'],
        axis=alt.Axis(title='Time')),
    y='Freq:Q',
    detail='id:N',
    opacity=alt.value(0.05),
    facet=alt.Facet('Cluster:N', columns=2)
).properties(width=400, height = 100)

alt.Chart(...)

### Question 4: For "Lost transit users", are they more likely to turn to private cars, or working from home?

### Driving alone?

The following figure shows the driving frequency per week before and after the pandemic. We can see that there isn't much change observed for each group. But to unravel the change in more detail, future analysis is needed.

In [109]:
Driving = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you.use.the.following.modes.of.transportation...Drive.alone.",
                     "In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you.use.the.following.modes.of.transportation...Drive.Alone.",
                     "Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will.use.the.following.modes.of.transportation...Drive.Alone."]]
Driving.columns = ["Before COVID", "During COVID", "After COVID"]

Driving.loc[(Driving.loc[:,"Before COVID"] == 'Never'),'Before COVID'] = 0
Driving.loc[(Driving.loc[:,"Before COVID"] == 'Once a month or less'),'Before COVID'] = 0.125
Driving.loc[(Driving.loc[:,"Before COVID"] == 'A few times a month'),'Before COVID'] = 0.625
Driving.loc[(Driving.loc[:,"Before COVID"] == '1-2 days a week'),'Before COVID'] = 1.5
Driving.loc[(Driving.loc[:,"Before COVID"] == '3-4 days a week'),'Before COVID'] = 3.5
Driving.loc[(Driving.loc[:,"Before COVID"] == 'Everyday'),'Before COVID'] = 7

Driving.loc[(Driving.loc[:,"During COVID"] == 'Never'),'During COVID'] = 0
Driving.loc[(Driving.loc[:,"During COVID"] == 'Once a month or less'),'During COVID'] = 0.125
Driving.loc[(Driving.loc[:,"During COVID"] == 'A few times a month'),'During COVID'] = 0.625
Driving.loc[(Driving.loc[:,"During COVID"] == '1-2 days a week'),'During COVID'] = 1.5
Driving.loc[(Driving.loc[:,"During COVID"] == '3-4 days a week'),'During COVID'] = 3.5
Driving.loc[(Driving.loc[:,"During COVID"] == 'Everyday'),'During COVID'] = 7

Driving.loc[(Driving.loc[:,"After COVID"] == 'Never'),'After COVID'] = 0
Driving.loc[(Driving.loc[:,"After COVID"] == 'Once a month or less'),'After COVID'] = 0.125
Driving.loc[(Driving.loc[:,"After COVID"] == 'A few times a month'),'After COVID'] = 0.625
Driving.loc[(Driving.loc[:,"After COVID"] == '1-2 days a week'),'After COVID'] = 1.5
Driving.loc[(Driving.loc[:,"After COVID"] == '3-4 days a week'),'After COVID'] = 3.5
Driving.loc[(Driving.loc[:,"After COVID"] == 'Everyday'),'After COVID'] = 7
Driving_cluster = pd.concat([Driving, pd.Series(Transit_kmeans.labels_)], axis=1)

Driving_cluster.columns = ["Before COVID", "During COVID", "After COVID", "Cluster"]

Driving1 = Driving_cluster.loc[Driving_cluster["Cluster"] == 0,:]
Driving2 = Driving_cluster.loc[Driving_cluster["Cluster"] == 1,:]
Driving3 = Driving_cluster.loc[Driving_cluster["Cluster"] == 2,:]
Driving4 = Driving_cluster.loc[Driving_cluster["Cluster"] == 3,:]

In [198]:


### WFH Group 3
Driving1_d = alt.Chart(Driving1).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

## WFH Group 4
Driving2_d = alt.Chart(Driving2).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

In [199]:
### WFH Group 3
Driving3_d = alt.Chart(Driving3).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

## WFH Group 4
Driving4_d = alt.Chart(Driving4).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

Driving3_d | Driving4_d

alt.HConcatChart(...)

In [200]:
Driving1_d | Driving2_d

alt.HConcatChart(...)

### Work from home?

Different from private cars, all groups have higher work from home frequency after the pandemic, especially for **lost transit riders** (top left) and **non-transit riders** (bottom right).

In [110]:
WFH = COVID_data[["In.a.typical.month.BEFORE.the.outbreak..e.g..in.January...how.often.did.you......work.from.home..",
                     "In.the.past.month..i.e..DURING.the.outbreak...how.often.did.you......work.from.home..",
                     "Sometime.in.the.future..when.COVID.19.will.no.longer.be.a.threat..how.often.do.you.think.you.will......work.from.home.."]]
WFH.columns = ["Before COVID", "During COVID", "After COVID"]

WFH.loc[(WFH.loc[:,"Before COVID"] == 'Never'),'Before COVID'] = 0
WFH.loc[(WFH.loc[:,"Before COVID"] == 'Once a month or less'),'Before COVID'] = 0.125
WFH.loc[(WFH.loc[:,"Before COVID"] == 'A few times a month'),'Before COVID'] = 0.625
WFH.loc[(WFH.loc[:,"Before COVID"] == '1-2 days a week'),'Before COVID'] = 1.5
WFH.loc[(WFH.loc[:,"Before COVID"] == '3-4 days a week'),'Before COVID'] = 3.5
WFH.loc[(WFH.loc[:,"Before COVID"] == 'Everyday'),'Before COVID'] = 7

WFH.loc[(WFH.loc[:,"During COVID"] == 'Never'),'During COVID'] = 0
WFH.loc[(WFH.loc[:,"During COVID"] == 'Once a month or less'),'During COVID'] = 0.125
WFH.loc[(WFH.loc[:,"During COVID"] == 'A few times a month'),'During COVID'] = 0.625
WFH.loc[(WFH.loc[:,"During COVID"] == '1-2 days a week'),'During COVID'] = 1.5
WFH.loc[(WFH.loc[:,"During COVID"] == '3-4 days a week'),'During COVID'] = 3.5
WFH.loc[(WFH.loc[:,"During COVID"] == 'Everyday'),'During COVID'] = 7

WFH.loc[(WFH.loc[:,"After COVID"] == 'Never'),'After COVID'] = 0
WFH.loc[(WFH.loc[:,"After COVID"] == 'Once a month or less'),'After COVID'] = 0.125
WFH.loc[(WFH.loc[:,"After COVID"] == 'A few times a month'),'After COVID'] = 0.625
WFH.loc[(WFH.loc[:,"After COVID"] == '1-2 days a week'),'After COVID'] = 1.5
WFH.loc[(WFH.loc[:,"After COVID"] == '3-4 days a week'),'After COVID'] = 3.5
WFH.loc[(WFH.loc[:,"After COVID"] == 'Everyday'),'After COVID'] = 7
WFH_cluster = pd.concat([WFH, pd.Series(Transit_kmeans.labels_)], axis=1)

WFH_cluster.columns = ["Before COVID", "During COVID", "After COVID", "Cluster"]

WFH1 = WFH_cluster.loc[WFH_cluster["Cluster"] == 0,:]
WFH2 = WFH_cluster.loc[WFH_cluster["Cluster"] == 1,:]
WFH3 = WFH_cluster.loc[WFH_cluster["Cluster"] == 2,:]
WFH4 = WFH_cluster.loc[WFH_cluster["Cluster"] == 3,:]

In [201]:
### WFH Group 1
WFH1_d = alt.Chart(WFH1).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

## WFH Group 2
WFH2_d = alt.Chart(WFH2).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

In [202]:
### WFH Group 3
WFH3_d = alt.Chart(WFH3).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

## WFH Group 4
WFH4_d = alt.Chart(WFH4).transform_fold(
    ['Before COVID',
     'After COVID'],
    as_ = ['Time', 'value']
).transform_density(
    density='value',
    bandwidth=0.1,
    groupby=['Time'],
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q'),
    alt.Row('Time:O')
).properties(width=350, height=40)

WFH3_d | WFH4_d

alt.HConcatChart(...)

In [203]:
WFH1_d| WFH2_d

alt.HConcatChart(...)

### Summary and future investigations

This assignment investigates the physical and virtual mobility change of residents in Puget Sound Region using a travel survey dataset. By applying a K-means clustering method, 4 groups of transit use patterns are identified. For **lost transit riders**, we found that their anticipated driving frequency does not change much, while their working from home frequency increases. Does that mean: those who are reluctant to return to transit after the pandemic, actually prefer working from home rather than switch to private cars?

In the future, there is a lot to be explored:

+ How much do they switch to private cars, work from home?

+ Is this good bad for future transit development and future sustainability?